# NASA Orbital Data Cleaning 

This notebook takes in raw data from NASA's Near Earth Object (NEO) database and cleans it.
<BR>
The purpose of this is to get the data into the desired format so that it could be run through visualisation code that I wrote in RStudio. 

In [1]:
#Importing relevant pacakges and loading in the data frame (df)

import pandas as pd
import numpy  as np

import re
from datetime import datetime

#import the data frame
df_raw = pd.read_csv('cneos_closeapproach_data.csv')
df_raw

,Object,Close-Approach (CA) Date,CA Distance Nominal (LD | au),CA Distance Minimum (LD | au),V relative (km/s),V infinity (km/s),H (mag),Diameter,Unnamed: 8
0,(2022 DX2),2022-Mar-13 04:32 ± < 00:01,15.69 | 0.04031,15.64 | 0.04019,12.69,12.68,25.0,26 m - 59 m,bK22D02X
1,(2022 DR3),2022-Mar-13 08:08 ± < 00:01,15.60 | 0.04008,15.51 | 0.03985,12.42,12.42,25.3,24 m - 53 m,bK22D03R
2,(2018 GY),2022-Mar-13 08:33 ± < 00:01,11.86 | 0.03048,11.86 | 0.03048,10.68,10.67,24.6,32 m - 71 m,bK18G00Y
3,(2022 ES3),2022-Mar-13 19:17 ± 00:04,0.87 | 0.00223,0.86 | 0.00222,18.37,18.30,27.1,10 m - 23 m,bK22E03S
4,(2022 EO4),2022-Mar-13 19:36 ± 00:09,2.75 | 0.00708,2.74 | 0.00705,15.64,15.61,25.4,22 m - 49 m,bK22E04O
...,...,...,...,...,...,...,...,...,...
12043,(2017 UH5),2200-Oct-29 07:16 ± 00:02,19.04 | 0.04891,19.03 | 0.04890,6.60,6.59,26.6,13 m - 28 m,bK17U05H
12044,475534 (2006 TS7),2200-Nov-01 07:43 ± < 00:01,4.47 | 0.01148,4.47 | 0.01148,17.73,17.72,21.2,150 m - 340 m,a0475534
12045,413577 (2005 UL5),2200-Nov-23 02:07 ± 00:10,5.74 | 0.01474,5.62 | 0.01445,18.17,18.16,20.2,240 m - 540 m,a0413577
12046,(2014 WT202),2200-Nov-23 03:36 ± 00:04,16.33 | 0.04197,16.23 | 0.04170,12.00,11.99,21.1,160 m - 360 m,bK14WK2T


# Initial Thoughts
<br>
Before begining the EDA (which will be done in an RStudio program) I need to clean the data formatting as it's not currently useful for analysis

* Close-Approach needs to be converted to data and split out the error
* Distance nominal needs to be split into a LD and AU column
* Diameter plit into a min and max columns

In [2]:
# print the columns so I can inspect what I currently have.
df_raw.columns

Index(['Object', 'Close-Approach (CA) Date', 'CA Distance Nominal (LD | au)',
       'CA Distance Minimum (LD | au)', 'V relative (km/s)',
       'V infinity (km/s)', 'H (mag)', 'Diameter', 'Unnamed: 8'],
      dtype='object')

In [3]:
#Beginning the Cleaning process.

# Create CA_Date_List
CA_Date_List = []
for i in range(0,len(df_raw)):
    CA_Date = df_raw['Close-Approach (CA) Date'][i]
    CA_Date = CA_Date[0:11]
    CA_Date_List.append(CA_Date)
    
# Create function to conver month name to num
def month_num(month_name):
    return {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9, 
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
    }[month_name]

#Create CA_Time_List
CA_Time_List = []
for i in range(0, len(df_raw)):
    CA_Time = df_raw['Close-Approach (CA) Date'][i]
    CA_Time = CA_Time[12:17]
    CA_Time_List.append(CA_Time)

#Set up the loop

month_format_list = []
for i in range(0, len(CA_Date_List)):
    x = CA_Date_List[i].split('-')
    month = x[1]
    month = str(month_num(month))
    formatted_month = x[0] +'/'+ month + '/'+ x[2] + ' ' + CA_Time_List[i]
    date_time_obj = datetime.strptime(formatted_month, '%Y/%m/%d %H:%M')
    
   
    month_format_list.append(date_time_obj)
month_format_list     



    
#Create CA_Time_Error_List
CA_Time_Error_List = []
for i in range(0, len(df_raw)):
    CA_Time_Error = df_raw['Close-Approach (CA) Date'][i]
    CA_Time_Error = CA_Time_Error[-5:]
    CA_Time_Error_List.append(CA_Time_Error)


In [4]:
#Create a loop to go through these

# Create CA Distance Nominal LD
CA_Dist_Nom_LD_List = []
CA_Dist_Nom_AU_List = []

for i in range(0,len(df_raw)):
    x = df_raw['CA Distance Nominal (LD | au)'][i]
    
    CA_Dist_Nom_LD = x.split(' | ')[0]
    CA_Dist_Nom_AU = x.split(' | ')[1]
    
    CA_Dist_Nom_LD_List.append(CA_Dist_Nom_LD)
    CA_Dist_Nom_AU_List.append(CA_Dist_Nom_AU)
    

# Repeat for minimum distance
CA_Dist_Min_LD_List = []
CA_Dist_Min_AU_List = []

for i in range(0,len(df_raw)):
    x = df_raw['CA Distance Minimum (LD | au)'][i]
    
    CA_Dist_Min_LD = x.split(' | ')[0]
    CA_Dist_Min_AU = x.split(' | ')[1]
    
    CA_Dist_Min_LD_List.append(CA_Dist_Min_LD)
    CA_Dist_Min_AU_List.append(CA_Dist_Min_AU)

In [5]:
# df_raw_diameter
df_raw['Diameter'] = df_raw['Diameter'].astype(str)

In [6]:
dia_min_list = []
dia_max_list = []

for i in range(0, len(df_raw)):
    y = df_raw['Diameter'][i]
    x = re.findall('[0-9.]+', y)
    if len(x) == 0:
        dia_min = 0
        dia_max = 0
    
    elif len(x) == 1:
        dia_min = x[0]
        dia_max = x[0]
    else:
        dia_min = x[0]
        dia_max = x[1]
        
    dia_min_list.append(dia_min)
    dia_max_list.append(dia_max)
    

dia_min_list_float = []
for i in (dia_min_list):
    dia_min_list_float.append(float(i))
    
dia_max_list_float = []
for i in dia_max_list:
    dia_max_list_float.append(float(i))

After running the above I can see that these two NEOs have no valid diameter for H (mag) and Diameter. I am comfortable excluding these as 2 samples will not be statistically relevant in this data set.

In [7]:

# 'V relative (km/s)',
#        'V infinity (km/s)', 'H (mag)'

#Move Obejct to the new df
Object_List = list(df_raw['Object'])

#Move v_rel to the new df
V_Rel = list(df_raw['V relative (km/s)'])
#Move v_inf to df
V_Inf = list(df_raw['V infinity (km/s)'])
#Move H (mag) to new df
H_Mag = list(df_raw['H (mag)'])

In [8]:
# Create the cleaned data frame (df_cleaned)

df_cleaned = pd.DataFrame(list(zip(Object_List, month_format_list, CA_Time_Error_List, CA_Dist_Nom_LD_List,
                                  CA_Dist_Nom_AU_List, CA_Dist_Min_LD_List, CA_Dist_Min_AU_List, dia_min_list_float,
                                  dia_max_list_float, V_Rel, V_Inf, H_Mag)),
                 columns = ['Object', 'Close Approach Date and Time', 'Close Approach Time Error',
                           'Close Approach Nominal Distance LD', 'Close Approach Nominal Distance AU',
                           'Close Approach Minimum Distance LD', 'Close Approach Minimum Distance AU', 
                           'Diameter Min (m)', 'Diameter Max (m)', 'Relative Velocity (km/s)', 'V Infinity (km/s)', 
                           'Absolute Magnitude (mag)'])
df_cleaned.head()

,Object,Close Approach Date and Time,Close Approach Time Error,Close Approach Nominal Distance LD,Close Approach Nominal Distance AU,Close Approach Minimum Distance LD,Close Approach Minimum Distance AU,Diameter Min (m),Diameter Max (m),Relative Velocity (km/s),V Infinity (km/s),Absolute Magnitude (mag)
0,(2022 DX2),2022-03-13 04:32:00,00:01,15.69,0.04031,15.64,0.04019,26.0,59.0,12.69,12.68,25.0
1,(2022 DR3),2022-03-13 08:08:00,00:01,15.60,0.04008,15.51,0.03985,24.0,53.0,12.42,12.42,25.3
2,(2018 GY),2022-03-13 08:33:00,00:01,11.86,0.03048,11.86,0.03048,32.0,71.0,10.68,10.67,24.6
3,(2022 ES3),2022-03-13 19:17:00,00:04,0.87,0.00223,0.86,0.00222,10.0,23.0,18.37,18.30,27.1
4,(2022 EO4),2022-03-13 19:36:00,00:09,2.75,0.00708,2.74,0.00705,22.0,49.0,15.64,15.61,25.4


In [9]:
# In the above process, some numerical values were converted to strings.
# The below code will convert them back to numeric values. 

df_cleaned['Close Approach Nominal Distance LD']  = pd.to_numeric(df_cleaned['Close Approach Nominal Distance LD'], 
                                                                  downcast = "float")
df_cleaned['Close Approach Nominal Distance AU']  =pd.to_numeric(df_cleaned['Close Approach Nominal Distance AU'], 
                                                                 downcast = "float")
df_cleaned['Close Approach Minimum Distance LD'] = pd.to_numeric(df_cleaned['Close Approach Minimum Distance LD'], 
                                                                 downcast = "float")
df_cleaned['Close Approach Minimum Distance AU']  =pd.to_numeric(df_cleaned['Close Approach Minimum Distance AU'],
                                                                  downcast = "float")

# Close approach year column
df_cleaned['Close Approach Date and Time'] = pd.DatetimeIndex(df_cleaned['Close Approach Date and Time']).year

In [10]:
df_cleaned.dtypes

Object                                 object
Close Approach Date and Time            int64
Close Approach Time Error              object
Close Approach Nominal Distance LD    float32
Close Approach Nominal Distance AU    float32
Close Approach Minimum Distance LD    float32
Close Approach Minimum Distance AU    float32
Diameter Min (m)                      float64
Diameter Max (m)                      float64
Relative Velocity (km/s)              float64
V Infinity (km/s)                     float64
Absolute Magnitude (mag)              float64
dtype: object

In [11]:
#Write to csv
df_cleaned.to_csv('neo_data.csv')

# Final Output

After writing the cleaned dataframe to a csv file, it is printed below to look at and visually inspect.

In [12]:
df_cleaned

,Object,Close Approach Date and Time,Close Approach Time Error,Close Approach Nominal Distance LD,Close Approach Nominal Distance AU,Close Approach Minimum Distance LD,Close Approach Minimum Distance AU,Diameter Min (m),Diameter Max (m),Relative Velocity (km/s),V Infinity (km/s),Absolute Magnitude (mag)
0,(2022 DX2),2022,00:01,15.690000,0.04031,15.640000,0.04019,26.0,59.0,12.69,12.68,25.0
1,(2022 DR3),2022,00:01,15.600000,0.04008,15.510000,0.03985,24.0,53.0,12.42,12.42,25.3
2,(2018 GY),2022,00:01,11.860000,0.03048,11.860000,0.03048,32.0,71.0,10.68,10.67,24.6
3,(2022 ES3),2022,00:04,0.870000,0.00223,0.860000,0.00222,10.0,23.0,18.37,18.30,27.1
4,(2022 EO4),2022,00:09,2.750000,0.00708,2.740000,0.00705,22.0,49.0,15.64,15.61,25.4
...,...,...,...,...,...,...,...,...,...,...,...,...
12043,(2017 UH5),2200,00:02,19.040001,0.04891,19.030001,0.04890,13.0,28.0,6.60,6.59,26.6
12044,475534 (2006 TS7),2200,00:01,4.470000,0.01148,4.470000,0.01148,150.0,340.0,17.73,17.72,21.2
12045,413577 (2005 UL5),2200,00:10,5.740000,0.01474,5.620000,0.01445,240.0,540.0,18.17,18.16,20.2
12046,(2014 WT202),2200,00:04,16.330000,0.04197,16.230000,0.04170,160.0,360.0,12.00,11.99,21.1
